<h1 style="color:#1398A1">TRAINING<h1>


<h3 style="color:#CCC229">LIBRAIRIES IMPORT<h3>


In [5]:
import sys
from pathlib import Path

src_path = Path("..")
sys.path.append(str(src_path))
%load_ext autoreload
%autoreload 2

from app.src.agent.utils.infos import list_snake_models
from app.src.agent.training.train import train_snake

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<h3 style="color:#CCC229">SOME TESTS<h3>

In [ ]:
# Example of listing all available Snake models
list_snake_models()

In [6]:
# Example of training a Snake agent using PPO algorithm
train_snake(timesteps=1_000_00, grid_size=8, n_envs=10, algorithm="PPO")

2026/01/25 23:38:26 INFO mlflow.tracking.fluent: Experiment with name 'Snake_8x8' does not exist. Creating a new experiment.


✨ Création d'un nouvel agent vierge sur grille 8x8.

Démarrage Run MLflow : NEW_TRAINING_20260125_233826_69b21c9f
Nouvel ID : 69b21c9f-56e7-433d-b9eb-fa60abc47c87
Using cpu device

Démarrage de l'entraînement pour 10000000 timesteps...
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 20.8     |
|    ep_rew_mean     | -1.01    |
| time/              |          |
|    fps             | 11369    |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 20480    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 20.3        |
|    ep_rew_mean          | -1.01       |
| time/                   |             |
|    fps                  | 2524        |
|    iterations           | 2           |
|    time_elapsed         | 16          |
|    total_timesteps      | 40960       |
| train/                  |             |
|   

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

2026/01/26 01:09:28 WARNING mlflow.pyfunc: Failed to infer model signature: Type hint <input: <class 'numpy.ndarray'>, output: None> cannot be used to infer model signature and input example is not provided, model signature cannot be inferred.
Successfully registered model 'Snake_8x8'.
Created version '1' of model 'Snake_8x8'.


Terminé ! Nouveau modèle disponible : 69b21c9f-56e7-433d-b9eb-fa60abc47c87


In [ ]:
# Example of resuming training from a previous model using its UUID
train_snake(timesteps=50000, n_envs=8, algorithm="PPO", base_uuid="a752e28b-b125-4a7a-a042-3fa15478e0b9")